In [5]:
import requests
import notify2
import logging
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
import plotly.express as px

class covid:
    
    def __init__(self, url):   
        self.url = url
    
    def logged_errors(self, msg):        
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
        file_handler = logging.FileHandler('covid.log')
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
        logger.error(msg)
        
    def show_notification(self, title, message, icon):
        try:
            notify2.init('app')
            notification = notify2.Notification(title, message, icon)
            notification.set_urgency(2)
            notification.show()
            self.logged_errors(message)
        except Exception:
            return
    
    def get_http_data(self):
        try:
            http_data = requests.get(self.url)
            if http_data.status_code != 200:
                title = "FAILED"
                msg = f"http get request failed with : {http_data.status_code}"
                icon = "dialog-information"
                self.show_notification(title, msg, icon)
                return
            else:
                return http_data
        except Exception:
            return
        
    def parse_http_data(self, http_data):
        global soup
        soup = BeautifulSoup(http_data.text, 'lxml')
        self.get_total_world_data()
        
    def get_total_world_data(self):
        
        # Get Total Cases in World
        total_numbers = soup.find_all('div', class_='maincounter-number')
        global world_cases
        world_cases = []
        for t in total_numbers:
            world_cases.append(t.span.text.strip())
        
        # Get Total Active and Closed Cases in World
        global active_closed_cases
        active_closed_cases = []
        total_active_closed = soup.find_all('div',class_='number-table-main')
        for t in total_active_closed:
            active_closed_cases.append(t.text)
            
        # Get mild condition patient count and critical patient count
        global mild_critical_cases
        mild_critical_cases = []
        total_mild_critical_cases = soup.find_all('span', class_='number-table')
        for m in total_mild_critical_cases:
            mild_critical_cases.append(m.text.strip())
        
        mild_critical_cases = mild_critical_cases[:2]
    
    def get_all_countries_data(self):
        all_countries_data_tbody = soup.find('tbody')
        all_countries_data = ''
        for tr in all_countries_data_tbody.find_all('tr')[8:]:
            for td in tr.find_all('td'):
                all_countries_data += td.text.strip() + '$'
            all_countries_data += '\n'
        all_countries_data = all_countries_data.split('\n')
        
        global all_countries_data_temp
        all_countries_data_temp = []
        len_ = len(all_countries_data)
        for data in all_countries_data[:len_-1]:
            all_countries_data_temp.append(data.split('$')[:11])
            
    
    def create_dataframe(self):
        global df
        df = pd.DataFrame(all_countries_data_temp)
        columns = ['country', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_recovered',
                  'active_cases', 'serious_critical', 'total_cases_per_1M', 'deaths_per_1M', 'total_tests']
        df.columns = columns
        df.replace('', 'nan', inplace=True)
        df.replace('N/A', 'nan', inplace=True)
        
        df.drop(['total_cases_per_1M', 'deaths_per_1M'], axis=True, inplace=True)
        
        #write to a file
        df.to_csv('covid.csv',index=False,header=True)
        
    def string_to_num(self, string):
        if string == 'nan':
            return 0
        elif string[0] == '+':
            string = string[1:]
            return int(''.join(string.split(',')))
        else:
            return int(''.join(string.split(',')))
    
    def change_cols_data_to_numeric(self):
        df['total_cases'] = df['total_cases'].apply(self.string_to_num)
        df['new_cases'] = df['new_cases'].apply(self.string_to_num)
        df['total_deaths'] = df['total_deaths'].apply(self.string_to_num)
        df['new_deaths'] = df['new_deaths'].apply(self.string_to_num)
        df['total_recovered'] = df['total_recovered'].apply(self.string_to_num)
        df['active_cases'] = df['active_cases'].apply(self.string_to_num)
        df['serious_critical'] = df['serious_critical'].apply(self.string_to_num)
        df['total_tests'] = df['total_tests'].apply(self.string_to_num)
        print(tabulate(df.head(10)))
            
    @staticmethod
    def plot_total_numbers():
        fig = plt.figure()
        ax = fig.add_subplot(111)
        world_cases_temp = []
        ax.set(title='Current Coronavirus Cases Stats')
        for c in world_cases:
            world_cases_temp.append(int((''.join(c.split(',')))))
        text = ['total_cases','total_deaths','total_recovered']
        ax.bar(text,world_cases_temp)
        plt.show()
    
    @staticmethod
    def plot_active_closed_cases():
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set(title='Total Coronavirus Cases distribution')
        active_closed_cases_temp = []
        for c in active_closed_cases:
            active_closed_cases_temp.append(int((''.join(c.split(',')))))
        text = ['active_cases','closed_cases']
        ax.bar(text,active_closed_cases_temp)
        plt.show()

    @staticmethod
    def plot_mild_critical_cases():
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set(title='Total Active Cases distribution')
        mild_critical_cases_temp = []
        for c in mild_critical_cases:
            mild_critical_cases_temp.append(int((''.join(c.split(',')))))
        text = ['mild_cases','critical_cases']
        ax.bar(text,mild_critical_cases_temp)
        plt.show()
    
    @staticmethod
    def plot_country_active_cases():
        fig = px.scatter(df.head(15), x='country', y='active_cases',
                        size='active_cases', color='country', hover_name='country', size_max=60)
        fig.show()
    
    @staticmethod
    def plot_country_death_cases():
        fig = px.scatter(df.head(15), x='country', y='total_deaths',
                        size='total_deaths', color='country', hover_name='country', size_max=60)
        fig.show()
            
if __name__ == "__main__":
    
    url = "https://www.worldometers.info/coronavirus/"
    c = covid(url)
    http_data = c.get_http_data()
    if http_data:
        c.parse_http_data(http_data)
        c.get_all_countries_data()
        c.create_dataframe()
        c.change_cols_data_to_numeric()
        #c.plot_total_numbers()
        #c.plot_active_closed_cases()
        #c.plot_mild_critical_cases()
        c.plot_country_active_cases()
        c.plot_country_death_cases()

-  -----------  ------  ----  -----  ---  -----  ------  -----  -------
0  USA          561159   859  22133   28  33122  505904  11770  2845135
1  Spain        169496  2665  17489  280  64727   87280   7371   600000
2  Italy        156363     0  19899    0  34211  102253   3343  1010193
3  France       132591     0  14393    0  27186   91012   6845   333807
4  Germany      127854     0   3022    0  64300   60532   4895  1317887
5  UK            88621  4342  11329  717      0   76948   1559   367667
6  Iran          73303  1617   4585  111  45983   22735   3877   275427
7  Turkey        56956     0   1198    0   3446   52312   1665   376100
8  Belgium       30589   942   3903  303   6707   19979   1234   102151
9  Netherlands   26551   964   2823   86    250   23478   1384   115992
-  -----------  ------  ----  -----  ---  -----  ------  -----  -------
